<img src="https://upload.wikimedia.org/wikipedia/fr/thumb/9/9e/Logo_ENSG_G%C3%A9omatique_2012.svg/220px-Logo_ENSG_G%C3%A9omatique_2012.svg.png" style="margin-left:auto; margin-right:auto"/>


<center> 
    <h1>ENSG : statistiques appliquées en cartographie</h1> 
    <h2>TD 3 : application de l'ensemble des connaissances</h2> 
    <h3>Florian Bayer, Sami Guembour, Janvier 2025</h3>
</center> 

<hr style="height: 2px; color:  #94bd13 ; background-color:  #94bd13; width: 100%; border: none;">

## Objectifs : 
- Le premier objectif de ce TD est d'appliquer les connaissances acquises durant le cours et les TD précédents
- Puis d'aborder une réflexion autours de l'analyse de données en cartographie avec :
    - la réalisation d'une ACP avec Python
    - la cartographie des résultats, toujours avec Python


## Contexte :
Dans le cadre d'une mission proposée par une ONG, on vous demande de réaliser **trois cartes**  illustrant le niveau de développement des pays Africains à la fin des années 1990.

Un jeu de données vous est fourni avec par pays, des informations sur :

<blockquote>
    
- DEN99: densité de population (hab/km²) en 1999
- URB00: part de la population urbaine dans la population totale (%) en 2000
- AGR00: part des agriculteurs dans la population active (%) en 2000
- JEU99:	part des moins de 15 ans dans la population totale (%) en 1999
- VIE99:	part des plus de 60 ans dans la population totale (%) en 1999
- TMI99:	taux de mortalité infantile (pour 1000 naissances) en 1999
- EVH99:	espérance de vie des hommes (ans) en 1999
- EVF99:	espérance de vie des femmes (ans) en 1999
- PNB97:	produit national humain en 1997
- PIB99:	produit intérieur humain en 1997
- ALP95:	taux d'alphabétisation en 1995
- SCO95:	taux de scolarisation en 1995
- ISF99:	indice synthétique de fécondité (nombre d'enfants par femme en âge de procréer) en 1999
    
</blockquote>
A noter que certains pays sont manquants

<hr style="height: 2px; color:  #94bd13 ; background-color:  #94bd13; width: 100%; border: none;">

# 1. Importation

Le code suivant permet d'installer si nécessaire les packages du TD

## 1.1 des packages

In [1]:
from warnings import simplefilter
simplefilter(action="ignore", category=FutureWarning)

import os
import pandas as pd

try:
    import sklearn
except:
    print ("scikit-learn non installé, installation ... ")
    os.system("conda install scikit-learn -y")
    import sklearn
finally:
    print("scikit-learn est installé")
    
try:
    import altair
except:
    print ("altair non installé, installation ... ")
    os.system("conda install altair -y")
    import altair
finally:
    print("altair est installé")

scikit-learn est installé
altair est installé


## 1.2 des données

Les données sont directement récupérées sur Git. Elles sont aussi disponibles dans le répertoire ```./data```

In [2]:
df_td3 = pd.read_table(
    r"https://raw.githubusercontent.com/fbxyz/ENSG_L1_cartostat/main/td/data/TD3_Afrique.csv",
    sep=",",
)
df_td3 = df_td3.sort_values("CODE_ISO3").copy().reset_index(drop=True) # Les données sont triées avec une copie du dataframe avant d'être réindexées
df_td3.head()

,CODE_ISO3,nom,DEN99,URB00,AGR00,JEU99,VIE99,TMI99,EVH99,EVF99,PNB97,PIB99,ALP95,SCO95,ISF99
0,AGO,Angola,10,34,72,48,3,125,45,48,260,816,42,30,6.8
1,BDI,Burundi,204,9,90,48,3,105,44,47,140,509,35,23,6.5
2,BEN,Bénin,55,42,54,49,3,94,51,55,380,1403,37,38,6.3
3,BFA,Burkina Faso,42,18,92,48,3,94,46,47,250,552,19,19,6.7
4,BWA,Botswana,3,74,44,39,5,56,40,41,3310,5933,70,71,4.1


<hr style="height: 2px; color:  #94bd13 ; background-color:  #94bd13; width: 100%; border: none;">

# 2. Analyse univarié des données avec Python

<blockquote style="color:#bc4749">
    
Réalisez l'analyse univariée des 4 données quantitatives que vous avez sélectionné : 
- forme et caractéristiques des séries
- valeurs centrales et dispersion des séries
- interprétation des résultats

Appuyez-vous sur les outils abordés dans le TD1. 

Organisez ce notebook afin de présenter vos résultats et interprétations. Les notebooks sont particulièrement adaptés pour présenter et discuter de résultats avec des collègues.
    
</blockquote>


<hr style="height: 2px; color:  #94bd13 ; background-color:  #94bd13; width: 100%; border: none;">

# 3. Cartographie des résultats avec Qgis

<blockquote style="color:#bc4749">
    
A l'aide de Qgis, réalisez une planche regroupant les 4 cartes. Utilisez le fichier de données csv ainsi que le fichier PAYS_AFRIQUE_4326.topojson du répertoire ```./data```
    
Attention, faites en sorte que ces cartes soient comparables. 

Appuyez-vous sur les outils abordés dans le TD2. 
    
</blockquote>

<blockquote style="color:#bc4749">
    
Exportez votre planche cartographique en image (png, jpg ou même svg). Uploader cette image dans un répertoire accessible par le notebook et ajoutez la à une balise html image dans le notebook :

```<img alt="" src="chemin_vers_ma_planche.png">```
</blockquote>



<hr style="height: 2px; color:  #94bd13 ; background-color:  #94bd13; width: 100%; border: none;">

# 4. ACP et cartographie avec Python

Trois cartes ne semblent pas suffisantes pour mesurer le niveau de développement des pays africains à la fin des années 1990.

L'Analyse en Composante Principale apparaît comme une excellente alternative pour répondre à la demande de l'ONG : 
- Elle permet de réorganiser l'information contenue dans le tableau de départ sous la forme de composantes/facteurs/axes
- Sous la forme de nouveaux indicateurs indépendants et hiérarchisés

Sa réalisation avec Python et la cartographie des résultats -toujours avec Python- sont présentées ci-dessous. Il s'agira pour vous d'interpréter  les résultats, tout en vous illustrant la puissance de Python en analyse de données.




<hr style="height: 1px; color:  #a7aeae ; background-color:  #a7aeae; width: 25%; border: none;">

## 4.1 Préparation de l'ACP

La librairie [scikit-learn](https://scikit-learn.org/stable/) est idéale pour réaliser une ACP. C'est **la** librairie majoritairement utilisée en Machine Learning.

Si ce domaine vous intéresse, je vous conseille cet excellent ouvrage : 

[<q> Müller, A. C., & Guido, S. (2016). Introduction to machine learning with Python: a guide for data scientists. " O'Reilly Media, Inc.". </q>](https://books.google.com/books?hl=fr&lr=&id=1-4lDQAAQBAJ&oi=fnd&pg=PP1&dq=Introduction+to+Machine+Learning+with+Python&ots=28nMKNNM0Y&sig=8GE7PzHFzbH80hQG6E6N2gXup5M)



In [3]:
from sklearn.decomposition import PCA  # l'ACP
from sklearn.preprocessing import (
    StandardScaler,  # Pour centrer-réduire les données afin de réaliser une ACP normée
)

Dans un premier temps, on sélectionne les données quantitatives à utiliser dans l'ACP.

Remarquez l'assignation à la variable X qui est une notation standard pour les inputs d'un modèle avec scikit-learn. 

Pour plus de simplicité, on se contente d'exclure les colonnes code et nom. Le résultat est un numpy array

In [4]:
X = df_td3.drop(columns=["CODE_ISO3", "nom"]).values
type(X)

numpy.ndarray

On fait de même pour le nom des colonnes

In [5]:
cols = df_td3.drop(columns=["CODE_ISO3", "nom"]).columns
cols

Index(['DEN99', 'URB00', 'AGR00', 'JEU99', 'VIE99', 'TMI99', 'EVH99', 'EVF99',
       'PNB97', 'PIB99', 'ALP95', 'SCO95', 'ISF99'],
      dtype='object')

Et pour les pays

In [6]:
code_iso3 = df_td3["CODE_ISO3"].values
code_iso3

array(['AGO', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD',
       'COG', 'DJI', 'DZA', 'EGY', 'ETH', 'GAB', 'GHA', 'GIN', 'GMB',
       'GNB', 'GNQ', 'KEN', 'LBY', 'MAR', 'MDG', 'MLI', 'MOZ', 'MRT',
       'MWI', 'NAM', 'NER', 'NGA', 'SDN', 'SEN', 'SLE', 'TCD', 'TGO',
       'TUN', 'TZA', 'UGA', 'ZAF', 'ZMB', 'ZWE'], dtype=object)

Les données à utiliser n'étant pas de même nature, il faut utiliser une ACP normée. Pour cela, on utilise StandardScaler() pour centrer réduire les données de X (cf. cours et TD1)

In [7]:
scaler = StandardScaler()  # On instancie le scaler
scaler.fit(X)
X_cr = scaler.transform(X)

Les données ont bien été centrée-réduites (moyenne =0, écart-type = 1)

In [8]:
pd.DataFrame(X_cr, columns=cols).describe().round(2)

,DEN99,URB00,AGR00,JEU99,VIE99,TMI99,EVH99,EVF99,PNB97,PIB99,ALP95,SCO95,ISF99
count,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00,42.00
mean,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00
std,1.01,1.01,1.01,1.01,1.01,1.01,1.01,1.01,1.01,1.01,1.01,1.01,1.01
min,-0.98,-1.81,-2.53,-2.58,-1.74,-1.92,-1.49,-1.68,-0.71,-0.92,-2.22,-1.48,-2.40
25%,-0.77,-0.62,-0.73,-0.19,-0.52,-0.76,-0.65,-0.47,-0.56,-0.71,-0.81,-0.77,-0.34
50%,-0.25,-0.10,0.21,0.21,-0.52,0.11,-0.36,-0.19,-0.46,-0.48,-0.01,-0.32,0.19
75%,0.38,0.45,0.84,0.61,0.70,0.81,0.30,0.39,-0.06,0.08,0.78,0.90,0.71
max,4.01,2.69,1.40,1.40,3.13,1.48,2.91,2.85,3.58,2.81,1.74,2.23,1.80


<hr style="height: 1px; color:  #a7aeae ; background-color:  #a7aeae; width: 25%; border: none;">

## 4.2 Calcul de l'ACP normée

Les données sont prêtes, l'ACP avec 13 composantes peut-être calculée.

On prépare une liste avec le nom des colonnes pour chaque composante. Cela facilitera la lecture des dataframes par la suite

In [9]:
n_composantes = 13
x_list = [f"c{i:02d}" for i in range(1, n_composantes + 1)] 
print(x_list)

['c01', 'c02', 'c03', 'c04', 'c05', 'c06', 'c07', 'c08', 'c09', 'c10', 'c11', 'c12', 'c13']


Puis on lance l'ACP sur les données centrée réduite X_cr

In [10]:
acp = PCA(n_components=n_composantes)
acp.fit(X_cr)

PCA(n_components=13)

<hr style="height: 1px; color:  #a7aeae ; background-color:  #a7aeae; width: 25%; border: none;">

## 4.3 Interprétation de l'ACP normée

Les calculs sont déjà finis... Il ne reste plus qu'à interpréter les résultats

### 4.3.1 Part de variance portée par chaque composante

Pour rappel, la part de variance (% de valeur propre / inertie) renseigne sur la quantité d'information du tableau de départ portée par chaque facteur

In [11]:
var = (acp.explained_variance_ratio_ * 100).round(2)
var_cum = var.cumsum().round()

df_var = pd.DataFrame({"Composante": x_list, "Var": var, "Var_cum": var_cum})
df_var

,Composante,Var,Var_cum
0,c01,58.75,59.0
1,c02,12.79,72.0
2,c03,8.35,80.0
3,c04,6.85,87.0
4,c05,3.39,90.0
5,c06,3.10,93.0
6,c07,2.42,96.0
7,c08,1.70,97.0
8,c09,1.12,98.0
9,c10,0.75,99.0


La part de variance sur chaque composante est généralement représentée avec un screeplot

Pour faire ce graphique, le package Altair va être utilisé à la place de Seaborn (pour des questions de goûts personnels...)

In [12]:
import altair as alt

screeplot_bar = (
    alt.Chart(
        df_var, title="Variance et variance cumulée pour chaque composante"
    )  # on instancie le graphique (chart)
    .mark_bar(size=15)  # on précise que l'on va faire un graphique avec des barres
    .encode(
        x=alt.X("Composante"),
        y=alt.Y("Var", title="% de la variance totale"),
        tooltip=[
            alt.Tooltip("Composante:N"),
            alt.Tooltip(
                "Var", format=".1f"
            ),  # on ajoute la possibilité d'affichier des informations au passage de la souris
        ],
    )
)

screeplot_cum = (
    alt.Chart(df_var)
    .mark_line(color="red")  # on ajoute ici un graphique en ligne
    .encode(
        x=alt.X("Composante"),
        y=alt.Y("Var_cum", title=""),
        tooltip=[
            alt.Tooltip("Composante:N"),
            alt.Tooltip("Var_cum", format=".1f"),
        ],
    )
)

alt.layer(screeplot_cum, screeplot_bar)  # on combine les deux graphiques

alt.LayerChart(...)

<blockquote style="color:#bc4749">
    
A la lumière de ce graphique (screeplot), que pouvez-vous en conclure sur la capacité de synthèse de votre ACP ?
    
</blockquote>

### 4.3.2 Coordonnées des variables sur les composantes

Il est maintenant nécessaire d'interpréter vos composantes. Pour cela, on regarde les coordonnées des variables sur les composantes, qui s'interprètent comme les corrélations de chaque variable sur les composantes

In [13]:
pcs = pd.DataFrame(acp.components_, columns=cols).round(2).T
pcs.columns = x_list
pcs

,c01,c02,c03,c04,c05,c06,c07,c08,c09,c10,c11,c12,c13
DEN99,0.08,0.34,0.71,0.39,0.04,0.41,0.07,-0.22,0.00,0.02,0.07,0.00,0.03
URB00,-0.27,0.00,-0.37,-0.25,0.05,0.67,-0.06,-0.46,0.16,0.18,-0.04,-0.10,0.01
AGR00,0.33,-0.07,-0.04,0.13,-0.33,-0.15,-0.00,-0.28,0.64,-0.29,-0.14,-0.37,-0.07
JEU99,0.31,-0.05,0.11,-0.22,0.52,-0.23,-0.06,-0.17,0.36,0.42,0.41,0.07,0.03
VIE99,-0.29,-0.15,-0.09,0.48,-0.11,-0.26,0.36,-0.18,0.04,0.61,-0.19,-0.04,-0.00
TMI99,0.29,-0.17,-0.21,0.15,-0.02,0.41,0.57,0.45,0.18,-0.01,0.25,0.10,0.13
EVH99,-0.27,0.49,-0.08,-0.09,0.14,-0.19,0.20,0.05,0.18,-0.13,-0.05,-0.14,0.71
EVF99,-0.26,0.50,-0.10,-0.08,0.11,-0.12,0.35,0.06,0.13,-0.12,0.18,-0.06,-0.67
PNB97,-0.30,-0.28,-0.08,0.37,0.22,-0.01,-0.15,-0.02,-0.13,-0.25,0.54,-0.48,0.05
PIB99,-0.32,-0.21,-0.01,0.27,0.29,-0.01,-0.09,-0.05,0.38,-0.35,-0.12,0.63,-0.02


<blockquote style="color:#bc4749">
    
Interprétez la composante 1 (c01). Que pouvez-vous en conclure par rapport à la demande de l'ONG ?
        
</blockquote>

<blockquote style="color:#bc4749">
    
Interprétez la composante 2 (c02). Etrange non ?
    
</blockquote>

<hr style="height: 1px; color:  #a7aeae ; background-color:  #a7aeae; width: 25%; border: none;">

## 4.4 Cartographie des composantes 1 et 2

Les composantes 1 et 2 étant interprétées, il est maintenant possible de les cartographier. 

### 4.4.1 Coordonnées des individus sur les composantes

Pour cela, on projette d'abord les individus sur les composantes. On obtient les coordonnées des individus sur ces composantes.

On ne garde que les deux premières composantes avec la propriété iloc de Pandas (seules les deux premières colonnes sont conservées)

In [14]:
X_proj = acp.transform(X_cr)
df_acp = pd.DataFrame(X_proj, columns=x_list).iloc[:, 0:2]
df_acp.insert(0, "CODE_ISO3", code_iso3)  # insertion en index 0 des codes iso des  pays
df_acp.head()

,CODE_ISO3,c01,c02
0,AGO,2.286678,-0.209791
1,BDI,3.392039,1.688386
2,BEN,1.038151,1.058158
3,BFA,3.036389,0.836286
4,BWA,-4.057903,-3.188678


On en profite pour discrétiser nos données. Pour cartographier les résultats d'une ACP, la méthode de la moyenne moyenne et de l'écart-type sont généralement utilisée

Pour ne pas complexifier ce TD, une discrétisation basée sur des quantiles est réalisée. Les résultats seront assez proches d'une discrétisation par moyenne et écart-type. 

pandas.qcut() est ici utilisé pour discrétiser à partir de quantiles. Il est possible d'utiliser pandas.cut() pour définir les bornes manuellement. Enfin, il existe des packages Python comme [mapclassify](https://pypi.org/project/mapclassify/) pour appliquer directement des méthodes de discrétisation, les bornes étant calculées automatiquement.  

In [15]:
def cut(x, q, c):
    """
    Retourne deux arrays avec les bornes de classes et des couleurs à partir d'un numpy array,
    de bornes de quantiles et d'un array de couleur
    """

    _cut = (
        pd.qcut(x, q=q, precision=1).astype(str).str.replace("(", "[", regex=False)
    )  # les libellés des bornes de classe
    _colors = pd.qcut(x, q=q, labels=c)  # les couleurs associées

    return _cut, _colors


colors = ['#d7191c','#fdae61','#ffffbf','#a6d96a','#1a9641']
quantiles = [0, 0.2, 0.4, 0.6, 0.80, 1]

df_acp["class_c01"], df_acp["colors_c01"] = cut(df_acp.c01, quantiles, colors)
df_acp["class_c02"], df_acp["colors_c02"] = cut(df_acp.c02, quantiles, colors)
df_acp.head()

,CODE_ISO3,c01,c02,class_c01,colors_c01,class_c02,colors_c02
0,AGO,2.286678,-0.209791,"[2.3, 4.2]",#1a9641,"[-0.9, -0.1]",#fdae61
1,BDI,3.392039,1.688386,"[2.3, 4.2]",#1a9641,"[1.2, 2.5]",#1a9641
2,BEN,1.038151,1.058158,"[0.2, 1.2]",#ffffbf,"[0.4, 1.2]",#a6d96a
3,BFA,3.036389,0.836286,"[2.3, 4.2]",#1a9641,"[0.4, 1.2]",#a6d96a
4,BWA,-4.057903,-3.188678,"[-6.6, -2.0]",#d7191c,"[-3.3000000000000003, -0.9]",#d7191c


## N.B. 
Le code ci-dessous vous montre comment créer une carte simple avec Altair. Des options plus avancées existent, mais ne seront pas abordées dans ce TD. Consultez le [site d'Altair](https://altair-viz.github.io/gallery/index.html#maps) pour plus de détails.

L'objectif est de vous montrer rapidement comment créer une carte avec Python. Pour plus de "liberté" dans la mise en page, vous pouvez utiliser [Geopandas](https://geopandas.org/en/stable/docs/user_guide/mapping.html) et Matplotlib. Seul, le premier permet de créer rapidement des cartes, sans pour autant atteindre un "bon niveau". Geopandas produisant un objet matplotlib, vous pouvez le modifier pour le rendre plus acceptable cartographiquement... Mais au prix d'étapes parfois complexes.

Pour cette illustration, nous nous contenterons donc d'Altair, qui est une bonne alternative : rapide à mettre en place pour un résultat presque satisfaisant

### 4.4.2 Chargement des données

Le fond de carte peut-être chargé directement à partir de l'URL du git du cours : 

In [16]:
url_basemap = "https://raw.githubusercontent.com/fbxyz/ENSG_L1_cartostat/main/td/data/PAYS_AFRIQUE_4326.topojson"  # Noter que le crs du fond est 4326, soit un système WGS84
basemap = alt.topo_feature(url=url_basemap, feature="afrique")

La carte est construite de la manière suivante avec Altair. Attention les commentaires à chaque ligne sont pour vous aider, il ne faut jamais faire ça dans votre code Python

In [17]:
def carto_acp(_basemap, _df, _composante, _main_title, _leg_title, _fields):
    
    """
    Création des cartes pour l'ACP du TD3. Join le fond de carte avec les données de l'ACP.
    Les bornes et couleurs de classes sont récupérées du dataframe.
    
    """

    _ind_class = f"class_{_composante}"
    _ind_colors = f"colors_{_composante}"

    _sort = _df.sort_values(_composante)
    _dom = _sort[_ind_class].unique()
    _col = list(_sort[_ind_colors].unique())

    # le fond de carte avec le type de graphique altair : geoshape
    _basemap = (
        alt.Chart(_basemap, title=_main_title)
        .mark_geoshape(  
            fill="white",
            stroke="#4d4d4d",
            strokeWidth=0.35,
        )
        .project(type="mercator")  # On projette le rendu
        .properties(width=600, height=600)  # la largeur et la hauteur de la carte en px
    )

    # l'acp à cartographier
    _acp_map = (
        _basemap.transform_lookup(  # transform_lookup permet de faire une jointure entre le topojson _basemap et le dataframe _df.
            lookup="properties.CODE_ISO3",  # dans _basemap, on cherche le champs CODE_ISO3 du noeud properties. Il servira à la jointure
            from_=alt.LookupData(
                data= _df, key="CODE_ISO3", fields=_fields
            ),  # left_join _df sur le champs COD_GEO, on ajoute les champs indicateurs : _fields
        )
        .mark_geoshape()
        .encode(
            color=alt.Color(
                f"{_ind_class}:O", # l'indicateur à cartographier.
                scale=alt.Scale(domain=_dom, range=_col),
                legend=alt.Legend(title=f"{_leg_title}"),
            ),  
            tooltip=[  # permet d'ajouter des informations sur le département au passage de la souris sur la carte
                alt.Tooltip("properties.nom:N", title="Pays"),
                alt.Tooltip(
                    f"{(_composante)}:O",format=".2f"
                ),
            ],
        )
    )
    
    # Ajout d'un texte descriptif sur l'ACP
    _text_acp = (
        alt.Chart()
        .mark_text(
            align="left", baseline="bottom", fontSize=11, color="#adb5bd"
        )
        .encode(
            x=alt.value(20),  # position en pixels
            y=alt.value(370), 
            text=alt.value(["Résultats d'une ACP normée menée de 42 pays",
                            "en fonction de critères démographiques et",
                            "socioéconomiques à la fin desannées 1990",
                            "Source : INED 2001"]),
        )
    )

    return (
        alt.layer(_basemap, _acp_map, _text_acp) # alt.layer combine les 3 graphiques en 1 seul
        .configure_legend(
            titleFontSize=14,  # la taille du titre de la légende
            labelFontSize=13,  # la taille des éléments de la légende
            titleColor="#495057",  # la couleur du titre de la légende
            labelColor="#495057",  # la couleur des éléments de la légende
            titleLimit=500,  # évite de couper le titre de la légende si trop long
            titlePadding=7,  # écart entre le titre de la légende et les éléments de la légende
            direction="horizontal",  # légende horizontale
            orient="bottom",  # légende placée en bas
            columns=10,  # nombre de colonnes max pour la légende
            columnPadding=10,  # espace entre les colonnes
            rowPadding=12,  # espace entre les lignes
            symbolType="square",  # type de caissons de légende
        )
        .configure_title(fontSize=16, color="#343a40")
    )


Il ne reste plus qu'à lancer la fonction carto_acp() :

In [18]:
composante = "c01" # pour cartographier la composante 2, changez en c02
map_title = [
    "Indicateur synthétique de niveau de développement",
    "des pays Africain à la fin des années 1990",
]

variance = df_var.query(f"Composante=='{composante}'").Var.values[0]
leg_title = f"Coordonnées sur le premier facteur ({variance}% de la variance totale)"
f = list(df_acp.columns)

carto_acp(
    _basemap=basemap,
    _df=df_acp,
    _composante=composante,
    _main_title=map_title,
    _leg_title=leg_title,
    _fields=f,
)

alt.LayerChart(...)

<blockquote style="color:#bc4749">
    
Faites la carte de la composante 2. Quel phénomène se cache derrière ?
    
</blockquote>

<hr style="height: 2px; color:  #94bd13 ; background-color:  #94bd13; width: 100%; border: none;">

# Conclusion du cours

Ce cours vous a permis d'aborder les statistiques en cartographie en fonction de deux axes : 
- l'analyse univariée des données, afin de vous aider à mieux comprendre le phénomène étudié
- la discrétisation, afin de transmettre efficacement le message cartographique lorsque vous traitez des données quantitatives de taux.

Vos connaissances en statistiques ont pu être appliquées à la géographie : 
- valeurs centrales et paramètres de dispersion
- tests de normalité
- tests comparaison de moyennes et de variances
- construction de graphiques associés aux calculs précédents

Des Notebook avec le langage Python ont été utilisés comme principaux outils d'analyse de données. Ce sont deux standards dans le monde professionnel.

Les différents TD vous ont permis d'appliquer vos connaissances, en vous rendant autonomes petit-à-petit.
TD 1 : 
- Prise en main de jupyter notebook et de python pour l'analyse univariée 
- Interprétation des résultats 
- Choix d'une méthode de discrétisation en fonction du public

TD 2 :
- Cartographie des résultats du TD 1 avec Qgis

TD 3 : 
- Sans aide, application des acquis des TD 1 et 2
- Introduction à l'analyse de données avec Python : réalisation d'une ACP
- Introduction à la cartographie avec Altair


Enfin, n'hésitez pas à me contacter si vous avez la moindre question.